# Tutorial 14: Custom Network with lane area detectors
In tutorial 7 was shown how to create a network with custom templates. Now this functionality shall be extended by the ability to add lane area detectors. 
Therefore this tutorial guides you through the process of creating a simulation with lane area detectors, as well as of how to create an environment, in which information is retrieved from the detectors for the reinforcement learning process.
The remainder of this tutorial is structured as follows:

* In Section 1 the classic set of aparameters is imported
* Section 2 guides you through the process of creating the pure simulation of the LuST Scenario as seen in tutorial 7 and  extend this by adding lane area detectors to the network
* Section 3 shows schematically how to use the lane area detectors for reinforcement learning

## 1. Importing Modules
Before we begin, let us import all relevant Flow parameters as we have done for previous tutorials. If you are unfamiliar with these parameters, you are encouraged to review tutorial 1.

In [ ]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# the base network class
from flow.networks import Network

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams

# create some default parameters parameters
HORIZON=1000
env_params = EnvParams(horizon=HORIZON)
initial_config = InitialConfig()

## 2. Create simulation with lane area detectors

### 2.1 Download LuST Scenario

If you did not already download the LuSTScenario in tutorial 7, do it now by running the following command:

git clone https://github.com/lcodeca/LuSTScenario.git

Once you have cloned the repository, please modify the code snippet below to match correct location of the repository's main directory.

In [ ]:
LuST_dir = "/path/to/LuSTScenario"

### 2.2 Add some InFlows for visualization purposes and later training

Start by creating the simulation paramaters. Set restart instance true, because otherwise inflows might slow down the process.

In [ ]:
from flow.core.params import SumoParams

sim_params = SumoParams(render=True, sim_step=1, restart_instance=True)

In [ ]:
vehicles=VehicleParams()

In [ ]:
from flow.core.params import InFlows

inflow = InFlows()

# chose examplary inflow-edges
inflow.add(veh_type="human",
           edge="-31846#2",
           vehs_per_hour=200)
inflow.add(veh_type="human",
           edge="-31850#0",
           vehs_per_hour=200)
inflow.add(veh_type="human",
           edge="-31854#1"

### 2.3 Creating an additional file with detector information

Now, additionally to the specifications "net", "vtype" and "rou" that were introduced in tutorial 7, a fourth file can be added, that is called "det".
An example xml could look like this:

<img src="img/detector_file.png">
<center><b>Figure 2</b>: Simulation of the LuST network </center>

Just check out the lane-ids of the LuST network, where you want to place your lane area detectors, and specify them like in the image shown above.

In [ ]:
import os

net_params = NetParams(
    inflows=inflow,
    template={
        # network geometry features
        "net": os.path.join(LuST_dir, "scenario/lust.net.xml"),
        # features associated with the properties of drivers
        "vtype": os.path.join(LuST_dir, "scenario/vtypes.add.xml"),
        # features associated with the routes vehicles take
        "rou": [os.path.join(LuST_dir, "scenario/DUARoutes/local.0.rou.xml"),
                os.path.join(LuST_dir, "scenario/DUARoutes/local.1.rou.xml"),
                os.path.join(LuST_dir, "scenario/DUARoutes/local.2.rou.xml")]
        # features associated with the detectors in the network
        # you need to create the file and specifiy its path here
        "det": os.path.join("path/to/detectors.add.xml")
    }
)

### 2.4Create custom network with lane area detectors

Finally, the fully imported simulation can be run as follows. 

In [ ]:
# create the network
network = Network(
    name="template",
    net_params=net_params,
    vehicles=vehicles
)

# create the environment for pure simulation
env = TestEnv(
    env_params=env_params,
    sim_params=sim_params,
    network=network
)

# run the simulation for 1000 steps
exp = Experiment(env=env)
_ = exp.run(1, 1000)

## 3. Doing reinforcement learning
Learning works in the same way as before. One needs to create an appropriate environment and just plug in the cutomized network.

### 3.1 Create environment
Here you can see a schematic environment, that makes use of lane area detectors. The information of the lane area detectors can be easily retrieved from self.k (kernel) as seen in earlier tutorials.

In [ ]:
from flow.envs import Env
class SimpleEnv(Env):
    
    def __init__(self, env_params, sim_params, network, simulator='traci'):
        super(SimpleEnv, self).__init__(env_params, sim_params, network, simulator='traci')
        self.det_ids = []
        self.veh_seen = []
        
        ...
    
    @property
    def action_space(self):
        ...
    
    @property
    def observation_space(self):
        number_of_ILs = len(self.k.lane_area_detector.get_ids())
        # nVehSeen reward shall be -nVehSeen, hence 1 variable per detector:
        number_of_signals = 1
        return Box(low=0, high=10000, shape=(number_of_signals * number_of_ILs,))
    
    def get_state(self):
        self.det_ids = self.k.lane_area_detector.get_ids()
        self.veh_seen = []
        for detector_id in self.det_ids:
            self.veh_seen.append(self.k.lane_area_detector.get_n_veh_seen(detector_id))
        return np.asarray(self.veh_seen)
    
    def _apply_rl_actions(self, rl_actions):
        ...
        
        
    def compute_reward(self, rl_actions, **kwargs):
        reward = -np.sum(self.veh_seen)
        return reward

### 3.2 Do the common process for RL training in flow
The following process should be familiar from the other tutorials. For example from tutorial 3 or 8

#### 3.2.1 Import custom environment
The custom environment needs to be important in order to work properly in flow.

In [ ]:
from flow.envs.simple_env import SimpleEnv
env_name = SimpleEnv

In [ ]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag="first_exp",
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=Network,
    # simulator that is used by the experiment
    ## the detectors are only usable for traci simulator right now! ##
    simulator='traci',
    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=VehicleParams(),
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

In [ ]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

In [ ]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)

In [ ]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

BATCH_SIZE = HORIZON

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = BATCH_SIZE  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1,  # number of iterations to stop after
        },
    },
})